In [6]:
import os
import pandas as pd
import numpy as np
from preprocess import *
import optuna
from sklearn.metrics import r2_score 
from optuna.samplers import TPESampler


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb
import xgboost as xgb

In [23]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

df_train = df_train.drop('Id',axis=1)
df_train = df_train.drop('parentspecies',axis=1)



X_train = df_train.loc[:,df_train.columns != 'pSat_Pa']
y_train = np.log10(df_train.loc[:, 'pSat_Pa'])
y_test = df_test

In [24]:
RANDOM_SEED = 42
train_x, test_x, train_y, test_y = train_test_split(X_train, y_train, test_size=0.25, random_state=RANDOM_SEED)

In [25]:

train_x.dtypes

MW                              float64
NumOfAtoms                        int64
NumOfC                            int64
NumOfO                            int64
NumOfN                            int64
NumHBondDonors                    int64
NumOfConf                         int64
NumOfConfUsed                     int64
C.C..non.aromatic.                int64
C.C.C.O.in.non.aromatic.ring      int64
hydroxyl..alkyl.                  int64
aldehyde                          int64
ketone                            int64
carboxylic.acid                   int64
ester                             int64
ether..alicyclic.                 int64
nitrate                           int64
nitro                             int64
aromatic.hydroxyl                 int64
carbonylperoxynitrate             int64
peroxide                          int64
hydroperoxide                     int64
carbonylperoxyacid                int64
nitroester                        int64
dtype: object

In [26]:
print(f"The dimension of train_x is {train_x.shape}")
print(f"The dimension of train_y is {train_y.shape}")
print(f"The dimension of test_x is {test_x.shape}")
print(f"The dimension of train_x is {test_y.shape}")

The dimension of train_x is (20360, 24)
The dimension of train_y is (20360,)
The dimension of test_x is (6787, 24)
The dimension of train_x is (6787,)


In [28]:

def objective(trial):
    
    params = {
        "n_estimators": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.1, log=True),
        "subsample": trial.suggest_float("subsample", 0.25, 0.75),
        "subsample_freq": 1,
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 0.5),
        "min_child_samples": trial.suggest_int("min_child_samples", 1, 50),
        "num_leaves": trial.suggest_int("num_leaves", 2, 2**10),
        "random_state": RANDOM_SEED
        
    }
    datas = lgb.Dataset(train_x, label=train_y)
    model = lgb.train(params, datas)

    predictions = model.predict(test_x)
    r2 = r2_score(test_y, predictions)
    return r2
    

# Remember to assign "lgbm-wine-1" as the study_name to your Optuna study. 
study_name = "lgbm-svp-1"

# For this assignment, it is enough to use a simple sqlite3 database for persisting study history
storage = "sqlite:///optuna.sqlite3"

# Create (and run) the study and record the history in the SQlite3 database file

study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED),
    study_name=study_name,
    storage=storage,
    load_if_exists=True
)
study.optimize(objective, n_trials=100)


print("Best R2", study.best_value)
print("Best params:", study.best_trial.params)

[I 2023-11-30 20:01:54,031] A new study created in RDB with name: lgbm-svp-1
/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000679 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 659
[LightGBM] [Info] Number of data points in the train set: 20360, number of used features: 23
[LightGBM] [Info] Start training from score -3.851741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


[I 2023-11-30 20:01:59,631] Trial 0 finished with value: 0.7324485362602042 and parameters: {'learning_rate': 0.005611516415334507, 'subsample': 0.7253571532049581, 'colsample_bytree': 0.3793972738151323, 'min_child_samples': 30, 'num_leaves': 161}. Best is trial 0 with value: 0.7324485362602042.
/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000700 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 659
[LightGBM] [Info] Number of data points in the train set: 20360, number of used features: 23
[LightGBM] [Info] Start training from score -3.851741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2023-11-30 20:02:04,174] Trial 1 finished with value: 0.6806990454304943 and parameters: {'learning_rate': 0.002051110418843397, 'subsample': 0.27904180608409973, 'colsample_bytree': 0.4397792655987208, 'min_child_samples': 31, 'num_leaves': 726}. Best is trial 0 with value: 0.7324485362602042.
/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000736 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 659
[LightGBM] [Info] Number of data points in the train set: 20360, number of used features: 23
[LightGBM] [Info] Start training from score -3.851741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2023-11-30 20:05:34,570] Trial 35 finished with value: 0.7249510912040071 and parameters: {'learning_rate': 0.016059761097420102, 'subsample': 0.5248282711685983, 'colsample_bytree': 0.4751689744218492, 'min_child_samples': 22, 'num_leaves': 865}. Best is trial 25 with value: 0.7363444503622264.
/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001906 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 659
[LightGBM] [Info] Number of data points in the train set: 20360, number of used features: 23
[LightGBM] [Info] Start training from score -3.851741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

[I 2023-11-30 20:05:42,297] Trial 36 finished with value: 0.7349438322770391 and parameters: {'learning_rate': 0.007406126591934742, 'subsample': 0.5634285522811873, 'colsample_bytree': 0.4173601488098583, 'min_child_samples': 47, 'num_leaves': 725}. Best is trial 25 with value: 0.7363444503622264.
/home/alex/.local/lib/python3.10/site-packages/lightgbm/engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.284159 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 663
[LightGBM] [Info] Number of data points in the train set: 20360, number of used features: 24
[LightGBM] [Info] Start training from score -3.851741
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

In [27]:
optuna.delete_study(study_name="lgbm-svp-1", storage="sqlite:///optuna.sqlite3")